In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plot
from sklearn.model_selection import train_test_split

import sklearn.gaussian_process as gp

from sklearn.model_selection import KFold

from __future__ import absolute_import, division, print_function, unicode_literals
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

In [3]:
from google.colab import files
src = list(files.upload().values())[0]
open('gp.py','wb').write(src)
import gp

%load gp.py
%run gp.py

Saving gp.py to gp (1).py


3899

####Importing the cifar dataset

In [46]:
fashion=keras.datasets.fashion_mnist

####Train-Test Split

In [47]:
(train_images, train_labels), (test_images, test_labels) = fashion.load_data()

In [38]:
train_images.shape

TensorShape([50000, 32, 32, 1])

No of models used- 1

Second model- Using two hidden layer

First we choose the no of nodes in the layer/layers randomly. We then check the accuracy of the model in train and test sets.

Now our target is to find the optimal no of nodes for a layer for each model.
We will use Bayes Optimization to find the optimal architechture for these two cases. We then check the accuracy in the train and test set by fitting models using optimal no of nodes.

So, steps- 
1. Fit model (By randomly choosing the no of nodes)
2. Evaluate the model on train and test set
   Now use Bayes Optimization to find optimal no of nodes.
3. Define a sample loss function and fix the bounds for the parameters
4. Find the optimal no of nodes
5. Fit another model using the optimal no of nodes
6. Evaluate the model on train and test sets



In [50]:
def create_model(n1,n2):
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(n1, activation='relu'),
    keras.layers.Dense(n2, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  return model

Here,
no of hidden layers=2
Activation function in hidden layer- Relu
No of Nodes in each hidden layer= 128

No of nodes in output layer= 10 (as total 10 classes are there)

In [51]:
model=create_model(130,130)
model.fit(train_images, train_labels, batch_size=512, epochs=30, verbose=0)

In [52]:
train_loss, train_acc = model.evaluate(train_images,  train_labels, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=0)
print("Train Accuracy",train_acc)
print("Test Accuracy",test_acc)

Train Accuracy 0.8584333062171936
Test Accuracy 0.8288000226020813


#####loss function to be used for optimization

In [53]:
def sample_loss(params):
  h1 = np.int(params[0])
  h2 = np.int(params[1])
  n_split=5
  cv_scores=[]
  for train_index,test_index in KFold(n_split).split(train_images):
    x_train,x_test=train_images[train_index],train_images[test_index]
    y_train,y_test=train_labels[train_index],train_labels[test_index]
    model_cv=create_model(h1,h2)
    model_cv.fit(x_train, y_train,batch_size=512,epochs=20,verbose=0)
    #print('Model evaluation ',model_cv.evaluate(x_test,y_test))
    cv_scores.append(model_cv.evaluate(x_test,y_test,verbose=0)[1])
  #print(cv_scores)
  return(np.array(cv_scores).mean())

####Bayesian Optimisation

In [ ]:
bound= np.array([[60,300],[60,300]])
xp, yp = bayesian_optimisation(n_iters=10, sample_loss=sample_loss,bounds=bound,n_pre_samples=10)

In [ ]:
xp_hat = np.round(xp[np.array(yp).argmax(), :])
print(np.round(xp_hat))

In [ ]:
model_opt=create_model(xp_hat[0],xp_hat[1])
model_opt.fit(train_images, train_labels, batch_size=512, epochs=30, verbose=0)

In [ ]:
train_loss, train_acc = model_opt.evaluate(train_images,  train_labels, verbose=0)
test_loss, test_acc = model_2_opt.evaluate(test_images,  test_labels, verbose=0)
print("Train Accuracy",train_acc)
print("Test Accuracy",test_acc)